# Illustrating of co-plotting `CATMAID` skeleton with flywire dataset

## Step 1: Import neccesary library modules now

In [ ]:
import navis
import fafbseg
import pymaid

In [ ]:
import pandas as pd
import numpy as np
import os
from copy import deepcopy

In [ ]:
from pyroglancer.localserver import startdataserver, closedataserver
from pyroglancer.flywire import flywireurl2dict, add_flywirelayer

## Step 2: Set configurations to fetch from data from `CATMAID`

In [ ]:
publicurl = 'https://fafb.catmaid.virtualflybrain.org/'
#privateurl = 'https://neuropil.janelia.org/tracing/fafb/v14'

In [ ]:
working_rm = pymaid.CatmaidInstance(publicurl, api_token=None, project_id = 1) # omit if not required

In [ ]:
# working_rm = pymaid.CatmaidInstance(privateurl,
#                             api_token=os.environ['TOKEN'],
#                             http_user=os.environ['authname'], # omit if not required
#                             http_password=os.environ['authpassword'], project_id = 1) # omit if not required

## Step 3: Get sample skids and neuropil meshes from `CATMAID`

In [ ]:
sample_skids = ['57323','57311','61221','755022','2863104'] #get some DA1 PNs..
#sample_skids = ['40637','27295','57311','2863104','57323']

In [ ]:
catmiad_neuronlist=pymaid.get_neurons(sample_skids,remote_instance = working_rm)

In [ ]:
vols = pymaid.get_volume(['v14.neuropil', 'AL_R'], color=(255, 0, 0, .2))

## Step 4: Transform them to Flywire space..

In [ ]:
flywire_neuron=navis.xform_brain(catmiad_neuronlist,source='FAFB14', target='FLYWIRE')

In [ ]:
flywirevol = {}
flywirevol['v14.neuropil']=navis.xform_brain(vols['v14.neuropil'],source='FAFB14', target='FLYWIRE')
flywirevol['AL_R']=navis.xform_brain(vols['AL_R'],source='FAFB14', target='FLYWIRE')

In [ ]:
flywirevol['v14.neuropil'].id = 200
flywirevol['AL_R'].id = 300
flywirevol

## Step 5: Start the dataserver to host precomputed data..

In [ ]:
startdataserver()

## Step 6: Get the url from flywire to append data onto..

In [ ]:
shorturl = 'https://ngl.flywire.ai/?json_url=https://globalv1.flywire-daf.com/nglstate/6027039450595328'
#This flywire segments has some stuff similar to the DA1 PNs, you will notice their closeness in the final plot

## Step 7: Add skids to flywire layers..

In [ ]:
tmpviewer = add_flywirelayer(flywireurl2dict(shorturl), layer_kws = {'type': 'skeletons','source': flywire_neuron})

## Step 8: Add neuropil meshes to flywire layers

In [ ]:
shorturl = tmpviewer

In [ ]:
tmpviewer = add_flywirelayer(flywireurl2dict(shorturl), layer_kws = {'type': 'volumes','source': [flywirevol['v14.neuropil'],
                                                                               flywirevol['AL_R']]})

# Screenshot of the flywire instance should like below..

### The grey mesh is the v14.1 neuropil, magenta is the Antenna lobe (Right), you can see the skeletons plotted in the skeleton layer, and corresponding meshes of the flywire dataset plotted as well

![alt text](catmaid2flywire.png "Title")

## Step 9: Add synapses and other annotations to flywire layers (not yet working)

In [ ]:
ngdict = flywireurl2dict(shorturl)

In [ ]:
tmpviewer = add_flywirelayer(ngdict, layer_kws = {'type': 'synapses','source': flywire_neuron,
                                                 'ngspace': 'Flywire'})

In [ ]:
temp_pts = pd.DataFrame([[123072, 47001, 3375]],columns=['x','y','z'])
temp_pts = pd.DataFrame([[123072, 47001, 3375], [120000, 17001, 3000]], columns=['x', 'y', 'z'])
temp_pts['description'] = ['center_pt','above_pt']

In [ ]:
#plot landmarks..
tmpviewer = add_flywirelayer(ngdict, layer_kws = {'type': 'points','name': 'landmarks',
                                                  'ngspace': 'Flywire',
                                                  'source': temp_pts,'color': 'red'})

In [ ]:
tmpviewer

## Step 10: Close the dataserver

In [ ]:
closedataserver()